In [21]:
## Solve Every Sudoku Puzzle

## See http://norvig.com/sudoku.html

## Throughout this program we have:
##   r is a row,    e.g. 'A'
##   c is a column, e.g. '3'
##   s is a square, e.g. 'A3'
##   d is a digit,  e.g. '9'
##   u is a unit,   e.g. ['A1','B1','C1','D1','E1','F1','G1','H1','I1']
##   grid is a grid,e.g. 81 non-blank chars, e.g. starting with '.18...7...
##   values is a dict of possible values, e.g. {'A1':'12349', 'A2':'8', ...}

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u])
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

################ Unit Tests ################

def test():
    "A set of tests that must pass."
    assert len(squares) == 81
    assert len(unitlist) == 27
    assert all(len(units[s]) == 3 for s in squares)
    assert all(len(peers[s]) == 20 for s in squares)
    assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                           ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                           ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
    assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                               'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                               'A1', 'A3', 'B1', 'B3'])
    print('All tests pass.')

################ Parse a Grid ################

def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    return values

def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    if len(chars) != 81: print(grid, chars, len(chars))
    assert len(chars) == 81
    return dict(zip(squares, chars))

################ Constraint Propagation ################

def assign(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in units[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
            if not assign(values, dplaces[0], d):
                return False
    return values

################ Display as 2-D grid ################

def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width) + ('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    print()

################ Search ################

def solve(grid): return search(parse_grid(grid))

def search(values):
    "Using depth-first search and propagation, try all possible values."
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in squares):
        return values ## Solved!
    ## Chose the unfilled square s with the fewest possibilities
    n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    for d in values[s]:
        result = search(assign(values.copy(), s, d))
        if result: return result

################ System test ################

import time

def solve_all(grids, name=''):
    """Attempt to solve a sequence of grids. Report results."""
    times, results = zip(*[time_solve(grid) for grid in grids])
    N = len(results)
    if N > 0:
        print("Solved %d of %d %s puzzles (avg %.2f secs (%d Hz), max %.2f secs)." % (
            sum(results), N, name, sum(times)/N, N/sum(times), max(times)))
            
def time_solve(grid):
    start = time.clock()
    values = solve(grid)
    t = time.clock()-start
    print(values.values())
    return (t, solved(values))

def solved(values):
    "A puzzle is solved if each unit is a permutation of the digits 1 to 9."
    def unitsolved(unit): return set(values[s] for s in unit) == set(digits)
    return values is not False and all(unitsolved(unit) for unit in unitlist)


grid1  = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid2  = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
hard1  = '.....6....59.....82....8....45........3........6..3.54...325..6..................'


In [22]:
solve_all([grid2])

dict_values(['4', '1', '7', '3', '6', '9', '8', '2', '5', '6', '3', '2', '1', '5', '8', '9', '4', '7', '9', '5', '8', '7', '2', '4', '3', '1', '6', '8', '2', '5', '4', '3', '7', '1', '6', '9', '7', '9', '1', '5', '8', '6', '4', '3', '2', '3', '4', '6', '9', '1', '2', '7', '5', '8', '2', '8', '9', '6', '4', '3', '5', '7', '1', '5', '7', '3', '2', '9', '1', '6', '8', '4', '1', '6', '4', '8', '7', '5', '2', '9', '3'])
Solved 1 of 1  puzzles (avg 0.01 secs (99 Hz), max 0.01 secs).


In [31]:
from random import randint, sample
from math import sqrt
import time

class SudokuSolver(object):
    def __init__(self, sudoku_size, init_values):
        self.sudoku_size = sudoku_size
        self.init_values = list(init_values)
        self.fixed_positions = []
        self.population = []
    def nodes(self):
        return self.sudoku_size * self.sudoku_size
    def kcolors(self):
        return self.sudoku_size
    def edges(self):
        return (self.nodes() * (self.kcolors()-1 + (self.kcolors()-(sqrt(self.kcolors()))) * 2)) / 2
    def population(self):
        return self.population
    def initPopulation(self, n_population):
        for i in range(0, n_population):
            solution = self.init_values[:]
            for i in range(0,len(solution)):
                c = solution[i]
                if c == 0 or c == '.':
                    solution[i] = randint(1,9)
                else:
                    solution[i] = int(solution[i])
                    self.fixed_positions.append(i)
            self.population.append(solution)
    def row_indexes(self, position):
        return range(int(position/self.sudoku_size)*9,int(position/self.sudoku_size)*9+9)
    def row_index(self, position):
        return int(position/self.sudoku_size)
    def col_index(self, position):
        return position%self.sudoku_size
    def col_indexes(self, position):
        return range(position%self.sudoku_size,73+(position%self.sudoku_size),9)
    def zone_number(self, position):
        row_index = self.row_index(position)
        col_index = self.col_index(position)
        if (row_index % 9 < 3):
            if (col_index % 9 < 3):
                return 0
            elif (col_index % 9 < 6):
                return 1
            else:
                return 2
        elif (row_index % 9 < 6):
            if (col_index % 9 < 3):
                return 3
            elif (col_index % 9 < 6):
                return 4
            else:
                return 5
        else:
            if (col_index % 9 < 3):
                return 6
            elif (col_index % 9 < 6):
                return 7
            else:
                return 8
    def zone_indexes(self, position):
        zone_number = self.zone_number(position)
        first = self.sudoku_size * int(zone_number/3)*3 + zone_number % 3 * 3
        indexes = []
        for i in range(0, 3):
            indexes.append(first) 
            indexes.append(1 + first)
            indexes.append(2 + first)
            first = first + self.sudoku_size
        return indexes
    def adyacents_indexes(self, position):
        indexes = set()
        [indexes.add(x) for x in self.col_indexes(position)]
        [indexes.add(x) for x in self.row_indexes(position)]
        [indexes.add(x) for x in self.zone_indexes(position)]
        indexes.remove(position)
        return indexes
    def fitness(self, solution):
        # Cantidad de ejes que unen nodos del mismo color
        bad_edges = 0
        for pos in range(0, self.nodes()):
            for ady in self.adyacents_indexes(pos):
                if (solution[ady] == solution[pos]):
                    bad_edges += 1
        return bad_edges / 2
    def mutate(self, solution):
        for i in range(0, len(solution)):
            if (randint(0, 1) and not(i in self.fixed_positions)):
                available_colors = set(range(1,10))
                for ady in self.adyacents_indexes(i):
                    if (solution[ady] in available_colors):
                        available_colors.remove(solution[ady])
                if (available_colors):
                    solution[i] = sample(available_colors, 1)[0]
                else:
                    solution[i] = randint(1,9)
        return solution
    def solve(self, n_population):
        start_time = time.time()
        max_iter = 50000
        self.initPopulation(n_population)
        best_fitness = self.edges()
        while (best_fitness > 0 and max_iter > 0):
            self.population = sorted(self.population, key=self.fitness)
            parent1 = self.population[0]
            parent2 = self.population[1]
            best_fitness = self.fitness(parent1)
            print("iteration %d, best_fitness %d" %(max_iter, best_fitness))
            for i in range(int(n_population/2), n_population):
                cut = randint(1, len(parent1)-2)
                child = parent1[0:cut] + parent2[cut:len(parent2)]
                self.population[i] = self.mutate(child)
            max_iter = max_iter - 1
        if (best_fitness == 0):
            print("--- Se resolvió el SUDOKU en %s segundos!!!! ---" % (time.time() - start_time))
        else:
            print("NO ANDUVO, se obtuvo un fitness de %d" % best_fitness)
        return self.population[0]

In [ ]:
init_values_easy = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
init_values_hard = '.....6....59.....82....8....45........3........6..3.54...325..6..................'
s = SudokuSolver(9, init_values_easy)
print(s.solve(8))

iteration 50000, best_fitness 79
iteration 49999, best_fitness 57
iteration 49998, best_fitness 51
iteration 49997, best_fitness 50
iteration 49996, best_fitness 42
iteration 49995, best_fitness 40
iteration 49994, best_fitness 33
iteration 49993, best_fitness 33
iteration 49992, best_fitness 33
iteration 49991, best_fitness 32
iteration 49990, best_fitness 32
iteration 49989, best_fitness 32
iteration 49988, best_fitness 32
iteration 49987, best_fitness 32
iteration 49986, best_fitness 32
iteration 49985, best_fitness 32
iteration 49984, best_fitness 32
iteration 49983, best_fitness 32
iteration 49982, best_fitness 31
iteration 49981, best_fitness 31
iteration 49980, best_fitness 31
iteration 49979, best_fitness 31
iteration 49978, best_fitness 30
iteration 49977, best_fitness 30
iteration 49976, best_fitness 30
iteration 49975, best_fitness 30
iteration 49974, best_fitness 30
iteration 49973, best_fitness 30
iteration 49972, best_fitness 30
iteration 49971, best_fitness 29
iteration 

iteration 49737, best_fitness 17
iteration 49736, best_fitness 17
iteration 49735, best_fitness 17
iteration 49734, best_fitness 17
iteration 49733, best_fitness 17
iteration 49732, best_fitness 17
iteration 49731, best_fitness 17
iteration 49730, best_fitness 17
iteration 49729, best_fitness 17
iteration 49728, best_fitness 17
iteration 49727, best_fitness 17
iteration 49726, best_fitness 17
iteration 49725, best_fitness 17
iteration 49724, best_fitness 17
iteration 49723, best_fitness 17
iteration 49722, best_fitness 17
iteration 49721, best_fitness 17
iteration 49720, best_fitness 17
iteration 49719, best_fitness 17
iteration 49718, best_fitness 17
iteration 49717, best_fitness 17
iteration 49716, best_fitness 17
iteration 49715, best_fitness 17
iteration 49714, best_fitness 17
iteration 49713, best_fitness 17
iteration 49712, best_fitness 17
iteration 49711, best_fitness 17
iteration 49710, best_fitness 17
iteration 49709, best_fitness 17
iteration 49708, best_fitness 17
iteration 

iteration 49471, best_fitness 14
iteration 49470, best_fitness 14
iteration 49469, best_fitness 14
iteration 49468, best_fitness 14
iteration 49467, best_fitness 14
iteration 49466, best_fitness 14
iteration 49465, best_fitness 14
iteration 49464, best_fitness 14
iteration 49463, best_fitness 14
iteration 49462, best_fitness 14
iteration 49461, best_fitness 14
iteration 49460, best_fitness 14
iteration 49459, best_fitness 14
iteration 49458, best_fitness 14
iteration 49457, best_fitness 14
iteration 49456, best_fitness 14
iteration 49455, best_fitness 14
iteration 49454, best_fitness 14
iteration 49453, best_fitness 14
iteration 49452, best_fitness 14
iteration 49451, best_fitness 14
iteration 49450, best_fitness 14
iteration 49449, best_fitness 14
iteration 49448, best_fitness 14
iteration 49447, best_fitness 14
iteration 49446, best_fitness 14
iteration 49445, best_fitness 14
iteration 49444, best_fitness 14
iteration 49443, best_fitness 14
iteration 49442, best_fitness 14
iteration 

iteration 49219, best_fitness 14
iteration 49218, best_fitness 14
iteration 49217, best_fitness 14
iteration 49216, best_fitness 14
iteration 49215, best_fitness 14
iteration 49214, best_fitness 14
iteration 49213, best_fitness 14
iteration 49212, best_fitness 14
iteration 49211, best_fitness 14
iteration 49210, best_fitness 14
iteration 49209, best_fitness 14
iteration 49208, best_fitness 14
iteration 49207, best_fitness 14
iteration 49206, best_fitness 14
iteration 49205, best_fitness 14
iteration 49204, best_fitness 14
iteration 49203, best_fitness 14
iteration 49202, best_fitness 14
iteration 49201, best_fitness 14
iteration 49200, best_fitness 14
iteration 49199, best_fitness 14
iteration 49198, best_fitness 14
iteration 49197, best_fitness 14
iteration 49196, best_fitness 14
iteration 49195, best_fitness 14
iteration 49194, best_fitness 14
iteration 49193, best_fitness 14
iteration 49192, best_fitness 14
iteration 49191, best_fitness 14
iteration 49190, best_fitness 14
iteration 

iteration 48952, best_fitness 14
iteration 48951, best_fitness 14
iteration 48950, best_fitness 14
iteration 48949, best_fitness 14
iteration 48948, best_fitness 14
iteration 48947, best_fitness 14
iteration 48946, best_fitness 14
iteration 48945, best_fitness 14
iteration 48944, best_fitness 14
iteration 48943, best_fitness 14
iteration 48942, best_fitness 14
iteration 48941, best_fitness 14
iteration 48940, best_fitness 14
iteration 48939, best_fitness 14
iteration 48938, best_fitness 14
iteration 48937, best_fitness 14
iteration 48936, best_fitness 14
iteration 48935, best_fitness 14
iteration 48934, best_fitness 14
iteration 48933, best_fitness 14
iteration 48932, best_fitness 14
iteration 48931, best_fitness 14
iteration 48930, best_fitness 14
iteration 48929, best_fitness 14
iteration 48928, best_fitness 14
iteration 48927, best_fitness 14
iteration 48926, best_fitness 14
iteration 48925, best_fitness 14
iteration 48924, best_fitness 14
iteration 48923, best_fitness 14
iteration 

...

iteration 27137, best_fitness 13
iteration 27136, best_fitness 13
iteration 27135, best_fitness 13
iteration 27134, best_fitness 13
iteration 27133, best_fitness 13
iteration 27132, best_fitness 13
iteration 27131, best_fitness 13
iteration 27130, best_fitness 13
iteration 27129, best_fitness 13
iteration 27128, best_fitness 13
iteration 27127, best_fitness 13
iteration 27126, best_fitness 13
iteration 27125, best_fitness 13
iteration 27124, best_fitness 13
iteration 27123, best_fitness 13
iteration 27122, best_fitness 13
iteration 27121, best_fitness 13
iteration 27120, best_fitness 13
iteration 27119, best_fitness 13
iteration 27118, best_fitness 13
iteration 27117, best_fitness 13
iteration 27116, best_fitness 13
iteration 27115, best_fitness 13
iteration 27114, best_fitness 13
iteration 27113, best_fitness 13
iteration 27112, best_fitness 13
iteration 27111, best_fitness 13
iteration 27110, best_fitness 13
iteration 27109, best_fitness 13
iteration 27108, best_fitness 13
iteration 

iteration 26876, best_fitness 13
iteration 26875, best_fitness 13
iteration 26874, best_fitness 13
iteration 26873, best_fitness 13
iteration 26872, best_fitness 13
iteration 26871, best_fitness 13
iteration 26870, best_fitness 13
iteration 26869, best_fitness 13
iteration 26868, best_fitness 13
iteration 26867, best_fitness 13
iteration 26866, best_fitness 13
iteration 26865, best_fitness 13
iteration 26864, best_fitness 13
iteration 26863, best_fitness 13
iteration 26862, best_fitness 13
iteration 26861, best_fitness 13
iteration 26860, best_fitness 13
iteration 26859, best_fitness 13
iteration 26858, best_fitness 13
iteration 26857, best_fitness 13
iteration 26856, best_fitness 13
iteration 26855, best_fitness 13
iteration 26854, best_fitness 13
iteration 26853, best_fitness 13
iteration 26852, best_fitness 13
iteration 26851, best_fitness 13
iteration 26850, best_fitness 13
iteration 26849, best_fitness 13
iteration 26848, best_fitness 13
iteration 26847, best_fitness 13
iteration 

iteration 26616, best_fitness 11
iteration 26615, best_fitness 11
iteration 26614, best_fitness 11
iteration 26613, best_fitness 11
iteration 26612, best_fitness 11
iteration 26611, best_fitness 11
iteration 26610, best_fitness 11
iteration 26609, best_fitness 11
iteration 26608, best_fitness 11
iteration 26607, best_fitness 11
iteration 26606, best_fitness 11
iteration 26605, best_fitness 11
iteration 26604, best_fitness 11
iteration 26603, best_fitness 11
iteration 26602, best_fitness 11
iteration 26601, best_fitness 11
iteration 26600, best_fitness 11
iteration 26599, best_fitness 11
iteration 26598, best_fitness 11
iteration 26597, best_fitness 11
iteration 26596, best_fitness 11
iteration 26595, best_fitness 11
iteration 26594, best_fitness 11
iteration 26593, best_fitness 11
iteration 26592, best_fitness 11
iteration 26591, best_fitness 11
iteration 26590, best_fitness 11
iteration 26589, best_fitness 11
iteration 26588, best_fitness 11
iteration 26587, best_fitness 11
iteration 

iteration 26352, best_fitness 11
iteration 26351, best_fitness 11
iteration 26350, best_fitness 11
iteration 26349, best_fitness 11
iteration 26348, best_fitness 11
iteration 26347, best_fitness 11
iteration 26346, best_fitness 11
iteration 26345, best_fitness 11
iteration 26344, best_fitness 11
iteration 26343, best_fitness 11
iteration 26342, best_fitness 11
iteration 26341, best_fitness 11
iteration 26340, best_fitness 11
iteration 26339, best_fitness 11
iteration 26338, best_fitness 11
iteration 26337, best_fitness 11
iteration 26336, best_fitness 11
iteration 26335, best_fitness 11
iteration 26334, best_fitness 11
iteration 26333, best_fitness 11
iteration 26332, best_fitness 11
iteration 26331, best_fitness 11
iteration 26330, best_fitness 11
iteration 26329, best_fitness 11
iteration 26328, best_fitness 11
iteration 26327, best_fitness 11
iteration 26326, best_fitness 11
iteration 26325, best_fitness 11
iteration 26324, best_fitness 11
iteration 26323, best_fitness 11
iteration 

iteration 26095, best_fitness 11
iteration 26094, best_fitness 11
iteration 26093, best_fitness 11
iteration 26092, best_fitness 11
iteration 26091, best_fitness 11
iteration 26090, best_fitness 11
iteration 26089, best_fitness 11
iteration 26088, best_fitness 11
iteration 26087, best_fitness 11
iteration 26086, best_fitness 11
iteration 26085, best_fitness 11
iteration 26084, best_fitness 11
iteration 26083, best_fitness 11
iteration 26082, best_fitness 11
iteration 26081, best_fitness 11
iteration 26080, best_fitness 11
iteration 26079, best_fitness 11
iteration 26078, best_fitness 11
iteration 26077, best_fitness 11
iteration 26076, best_fitness 11
iteration 26075, best_fitness 11
iteration 26074, best_fitness 11
iteration 26073, best_fitness 11
iteration 26072, best_fitness 11
iteration 26071, best_fitness 11
iteration 26070, best_fitness 11
iteration 26069, best_fitness 11
iteration 26068, best_fitness 11
iteration 26067, best_fitness 11
iteration 26066, best_fitness 11
iteration 

In [23]:
['4', '1', '7', '3', '6', '9', '8', '2', '5', '6', '3', '2', '1', '5', '8', '9', '4', '7', '9', '5', '8', '7', '2', '4', '3', '1', '6', '8', '2', '5', '4', '3', '7', '1', '6', '9', '7', '9', '1', '5', '8', '6', '4', '3', '2', '3', '4', '6', '9', '1', '2', '7', '5', '8', '2', '8', '9', '6', '4', '3', '5', '7', '1', '5', '7', '3', '2', '9', '1', '6', '8', '4', '1', '6', '4', '8', '7', '5', '2', '9', '3']

['4',
 '1',
 '7',
 '3',
 '6',
 '9',
 '8',
 '2',
 '5',
 '6',
 '3',
 '2',
 '1',
 '5',
 '8',
 '9',
 '4',
 '7',
 '9',
 '5',
 '8',
 '7',
 '2',
 '4',
 '3',
 '1',
 '6',
 '8',
 '2',
 '5',
 '4',
 '3',
 '7',
 '1',
 '6',
 '9',
 '7',
 '9',
 '1',
 '5',
 '8',
 '6',
 '4',
 '3',
 '2',
 '3',
 '4',
 '6',
 '9',
 '1',
 '2',
 '7',
 '5',
 '8',
 '2',
 '8',
 '9',
 '6',
 '4',
 '3',
 '5',
 '7',
 '1',
 '5',
 '7',
 '3',
 '2',
 '9',
 '1',
 '6',
 '8',
 '4',
 '1',
 '6',
 '4',
 '8',
 '7',
 '5',
 '2',
 '9',
 '3']